In [12]:
import numpy as np
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from matplotlib import cm 

from multiprocessing import Pool
from subprocess import call, STDOUT

from scipy import signal
from scipy.stats import norm, uniform

In [13]:
plt.rcParams.update({'font.size':14})

In [14]:
par = {'kappaX':1.2,
       'kappaY':0.9,
       'theta':1,
       'nA':3,
       'nB':3,
       'alpha':10,
       'betaBX':200,
       'betaFX':12,
       'betaBY':400,
       'betaFY':10,
       'deltaX':1.,
       'deltaY':0.05,}

In [15]:
tSpan = np.linspace(0,50,10001)

In [16]:
def H(x,kappa,theta,N):
    
    prob=np.ones([1,N+1])
    
    for i in range(1,N+1):
        prob[0,i] = prob[0,i-1] * kappa * x  / theta 
    
    total=0
    
    for i in range(N+1):
        total += prob[0,i]
    
    return prob[0][N]/total


def RHS(t,z,par):
    
    x,y  = z
    
    dxdt =  par['betaFX'] + (par['betaBX']-par['betaFX'])*H(x,par['kappaX'],par['theta'],3)-par['deltaX']*x - par['alpha']*x*y
    dydt =  par['betaFY'] + (par['betaBY']-par['betaFY'])*H(x,par['kappaY'],par['theta'],3)-par['deltaY']*y - par['alpha']*x*y
    
    return np.array([dxdt,dydt])

In [17]:
reCalculate = True

if reCalculate:
    
    betaFXgrid = np.linspace(5, 30, 301)
    betaBXgrid = np.linspace(150, 320, 301)
    
    betaBXgrid,betaFXgrid = np.meshgrid(betaBXgrid,betaFXgrid)
    
    betaBXgrid=betaBXgrid.reshape(np.size(betaBXgrid))
    betaFXgrid=betaFXgrid.reshape(np.size(betaFXgrid))
    
    fullOutput = np.zeros((np.size(betaBXgrid), 4))
    
    fullOutput[:,0] = betaFXgrid[:]
    fullOutput[:,1] = betaBXgrid[:]
    fullOutput[:,2] = 0
    fullOutput[:,3] = np.nan
    
    np.savez('detLandscape', fullOutput=fullOutput)

In [18]:
populationN = 100
poolN = populationN

In [19]:
def compileParList(par):
    
    populationN = len(par)
    
    parList = np.zeros((populationN, 3))
    
    parList[:,0] = range(populationN)
    parList[:,1] = par[:,0]
    parList[:,2] = par[:,1]
    
    return parList

In [20]:
def batchRun(parList,tSpan):
    
    tN = len(tSpan)
    populationN = len(parList)
    
    output = np.zeros((populationN, tN))
    
    counter = 0
    
    for _,betaFX,betaBX in parList:
        
        #print(counter)
        par['betaFX'] = betaFX
        par['betaBX'] = betaBX

        sol =  solve_ivp(fun=lambda t,z: RHS(t,z,par), t_span=(tSpan[0],tSpan[-1]), y0=(0.1,0.0), t_eval=tSpan, method='LSODA', rtol=1e-5, atol=1e-5)
        
        output[counter, :]  = sol.y[0,:]
        
        counter += 1

    return output

In [21]:
def evaluatePeak(solutions,dt):
    
    output = []
    
    output_fullPSD = []
    
    populationN = len(solutions)
    
    for i in range(populationN):
    
        data = solutions[i,:]
        x=data[5000:]

        x=(x-np.mean(x))
        fs = 1/dt
        freqTemp, psdTemp = signal.welch(x, fs, nperseg=256, nfft=len(x), window='boxcar', noverlap=0)

        maxDensity = np.amax(psdTemp)
        index = np.where(psdTemp==maxDensity)[0][0]
        maxFrequency = freqTemp[index]
        
        if maxDensity < 1E-4:
            
            maxDensity = 0    
            maxFrequency = 0
            
        output.append([maxFrequency, maxDensity])
        
        output_fullPSD.append(np.vstack((freqTemp, psdTemp)))

    return np.array(output),  np.array(output_fullPSD)

In [22]:
def evaluateFitness(parList,tSpan):
    
    populationN = len(parList)
    
    
    solutions = batchRun(parList,tSpan)
    
    
    l,_ = evaluatePeak(solutions, tSpan[1]-tSpan[0])
    
    # combine peak frequency and peak power
    targetF = 1.5

    C0 = 0
    C1 = 1
    
    return l[:,1]/(1+l[:,1])/(1+(l[:,0]-targetF)**2)

In [23]:
flag = True

while flag:
    
    fullOutput = np.load('detLandscape.npz', allow_pickle=True)['fullOutput']
    
    if np.sum(fullOutput[:,2])<len(fullOutput):
        
        onset = np.where(fullOutput[:,2]==0)[0][0]
        
        offset = min(onset+100, len(fullOutput))
        
        parList = compileParList(fullOutput[onset:offset,:2])
        
        print('Evaluate '+str(onset)+' to '+str(offset)+' out of '+str(len(fullOutput)))
        
        fitness = evaluateFitness(parList,tSpan)
        
        print('Done')
        
        fullOutput[onset:offset, 2]=1.0
        
        fullOutput[onset:offset, 3]=fitness
        
        np.savez('detLandscape', fullOutput=fullOutput)
                
    else:
        
        print('all done')
        flag = False
        

Evaluate 0 to 100 out of 90601
Done
Evaluate 100 to 200 out of 90601
Done
Evaluate 200 to 300 out of 90601
Done
Evaluate 300 to 400 out of 90601
Done
Evaluate 400 to 500 out of 90601
Done
Evaluate 500 to 600 out of 90601
Done
Evaluate 600 to 700 out of 90601
Done
Evaluate 700 to 800 out of 90601
Done
Evaluate 800 to 900 out of 90601
Done
Evaluate 900 to 1000 out of 90601
Done
Evaluate 1000 to 1100 out of 90601
Done
Evaluate 1100 to 1200 out of 90601
Done
Evaluate 1200 to 1300 out of 90601
Done
Evaluate 1300 to 1400 out of 90601
Done
Evaluate 1400 to 1500 out of 90601
Done
Evaluate 1500 to 1600 out of 90601
Done
Evaluate 1600 to 1700 out of 90601
Done
Evaluate 1700 to 1800 out of 90601
Done
Evaluate 1800 to 1900 out of 90601
Done
Evaluate 1900 to 2000 out of 90601
Done
Evaluate 2000 to 2100 out of 90601
Done
Evaluate 2100 to 2200 out of 90601
Done
Evaluate 2200 to 2300 out of 90601
Done
Evaluate 2300 to 2400 out of 90601
Done
Evaluate 2400 to 2500 out of 90601
Done
Evaluate 2500 to 2600

Done
Evaluate 20100 to 20200 out of 90601
Done
Evaluate 20200 to 20300 out of 90601
Done
Evaluate 20300 to 20400 out of 90601
Done
Evaluate 20400 to 20500 out of 90601
Done
Evaluate 20500 to 20600 out of 90601
Done
Evaluate 20600 to 20700 out of 90601
Done
Evaluate 20700 to 20800 out of 90601
Done
Evaluate 20800 to 20900 out of 90601
Done
Evaluate 20900 to 21000 out of 90601
Done
Evaluate 21000 to 21100 out of 90601
Done
Evaluate 21100 to 21200 out of 90601
Done
Evaluate 21200 to 21300 out of 90601
Done
Evaluate 21300 to 21400 out of 90601
Done
Evaluate 21400 to 21500 out of 90601
Done
Evaluate 21500 to 21600 out of 90601
Done
Evaluate 21600 to 21700 out of 90601
Done
Evaluate 21700 to 21800 out of 90601
Done
Evaluate 21800 to 21900 out of 90601
Done
Evaluate 21900 to 22000 out of 90601
Done
Evaluate 22000 to 22100 out of 90601
Done
Evaluate 22100 to 22200 out of 90601
Done
Evaluate 22200 to 22300 out of 90601
Done
Evaluate 22300 to 22400 out of 90601
Done
Evaluate 22400 to 22500 out o

Done
Evaluate 39700 to 39800 out of 90601
Done
Evaluate 39800 to 39900 out of 90601
Done
Evaluate 39900 to 40000 out of 90601
Done
Evaluate 40000 to 40100 out of 90601
Done
Evaluate 40100 to 40200 out of 90601
Done
Evaluate 40200 to 40300 out of 90601
Done
Evaluate 40300 to 40400 out of 90601
Done
Evaluate 40400 to 40500 out of 90601
Done
Evaluate 40500 to 40600 out of 90601
Done
Evaluate 40600 to 40700 out of 90601
Done
Evaluate 40700 to 40800 out of 90601
Done
Evaluate 40800 to 40900 out of 90601
Done
Evaluate 40900 to 41000 out of 90601
Done
Evaluate 41000 to 41100 out of 90601
Done
Evaluate 41100 to 41200 out of 90601
Done
Evaluate 41200 to 41300 out of 90601
Done
Evaluate 41300 to 41400 out of 90601
Done
Evaluate 41400 to 41500 out of 90601
Done
Evaluate 41500 to 41600 out of 90601
Done
Evaluate 41600 to 41700 out of 90601
Done
Evaluate 41700 to 41800 out of 90601
Done
Evaluate 41800 to 41900 out of 90601
Done
Evaluate 41900 to 42000 out of 90601
Done
Evaluate 42000 to 42100 out o

Done
Evaluate 59300 to 59400 out of 90601
Done
Evaluate 59400 to 59500 out of 90601
Done
Evaluate 59500 to 59600 out of 90601
Done
Evaluate 59600 to 59700 out of 90601
Done
Evaluate 59700 to 59800 out of 90601
Done
Evaluate 59800 to 59900 out of 90601
Done
Evaluate 59900 to 60000 out of 90601
Done
Evaluate 60000 to 60100 out of 90601
Done
Evaluate 60100 to 60200 out of 90601
Done
Evaluate 60200 to 60300 out of 90601
Done
Evaluate 60300 to 60400 out of 90601
Done
Evaluate 60400 to 60500 out of 90601
Done
Evaluate 60500 to 60600 out of 90601
Done
Evaluate 60600 to 60700 out of 90601
Done
Evaluate 60700 to 60800 out of 90601
Done
Evaluate 60800 to 60900 out of 90601
Done
Evaluate 60900 to 61000 out of 90601
Done
Evaluate 61000 to 61100 out of 90601
Done
Evaluate 61100 to 61200 out of 90601
Done
Evaluate 61200 to 61300 out of 90601
Done
Evaluate 61300 to 61400 out of 90601
Done
Evaluate 61400 to 61500 out of 90601
Done
Evaluate 61500 to 61600 out of 90601
Done
Evaluate 61600 to 61700 out o

Done
Evaluate 78900 to 79000 out of 90601
Done
Evaluate 79000 to 79100 out of 90601
Done
Evaluate 79100 to 79200 out of 90601
Done
Evaluate 79200 to 79300 out of 90601
Done
Evaluate 79300 to 79400 out of 90601
Done
Evaluate 79400 to 79500 out of 90601
Done
Evaluate 79500 to 79600 out of 90601
Done
Evaluate 79600 to 79700 out of 90601
Done
Evaluate 79700 to 79800 out of 90601
Done
Evaluate 79800 to 79900 out of 90601
Done
Evaluate 79900 to 80000 out of 90601
Done
Evaluate 80000 to 80100 out of 90601
Done
Evaluate 80100 to 80200 out of 90601
Done
Evaluate 80200 to 80300 out of 90601
Done
Evaluate 80300 to 80400 out of 90601
Done
Evaluate 80400 to 80500 out of 90601
Done
Evaluate 80500 to 80600 out of 90601
Done
Evaluate 80600 to 80700 out of 90601
Done
Evaluate 80700 to 80800 out of 90601
Done
Evaluate 80800 to 80900 out of 90601
Done
Evaluate 80900 to 81000 out of 90601
Done
Evaluate 81000 to 81100 out of 90601
Done
Evaluate 81100 to 81200 out of 90601
Done
Evaluate 81200 to 81300 out o